## Czyszczenie danych i przygotowanie zbioru do analizy Michał Suchan

### Treść zadania

##### Dane: czyszczenie i przygotowanie do analizy

Uwaga W tym zadaniu używamy skryptów (kodów), nie korzystamy z edytorów i arkuszy. Ze względu na wygodę i sprawne przeprowadzenie ćwiczenia zbiór na którym pracujecie nie jest duży - ale proszę sobie wyobrazić, ze macie wykonać czyszczenie zbioru mającego kilkanaście- kilkaset tysięcy rekordów - i Wasze skrypty/kody mają zadziałać i na dużo większym zbiorze niż ten, na którym pracujecie. Potencjalna skalowalność (łatwa adaptacja) waszego kodu będzie jednym z kryteriów oceny.

#### Przygotowanie pracy
Pracę można wykonać w R lub Python (rekomendowany Python+pandas/numpy)

1. Pobierz zbiór danych TitanicMess.tsv. Jest to plik którego separatorem jest tabulacja
2. Przygotuj skoroszyt Jupyter - w którym będziesz dokumentować poszczególne kroki analizy a następnie czyszczenia zbioru
3. Zapoznaj się ze zbiorem - przyglądnij mu się. Zidentyfikuj i opisz problemy z jakością danych które znalazłeś (aś) w poszczególnych kolumnach, w całym zbiorze. Opisz jakie problemy zidentyfikowałas(eś) i pokaz gdzie je widać albo na podstawie czego o nich wnioskujesz.
4. Wyczyść zbiór, tzn usuń problemy które znalazłeś/aś. Opisz krótko co i dlaczego robisz - i jaki efekt masz nadzieję osiągnąć. No i .. pokaż że się udało, bo problemu już nie ma.
5. Zapisz oczyszczony zbiór pod nazwą TitanicCleaned.tsv

##### Przesłanie - udostępnienie pracy
1. Umieść pliki: skoroszyt Jupyter z kodem i opisami, plik TitanicMess i TitanicCleaned w repozytorium git współpracującym z Binder
2. Podłącz skoroszyt do Binder-a. Sprawdź, czy skoroszyt działa i czy można z niego korzystać - uruchamiać kod
3. W polu tekstowym umieść linki zarówno do repozytorium jak i do uruchomionego w Binder Twojego notatnika Jupyter aby umożliwić zapoznanie się z Twoim kodem i opisem procesu czyszczenia danych.

### Rozwiązanie zadania

Skoroszyt Jupyter oparto o Pythona w wersji 3.

###### Wczytanie danych

Do wczytania danych posłuży biblioteka pandas. Plik TitanicMess.tsv powinien być wcześniej pobrany oraz umieszczony w folderze, gdzie znajduje się Notebook. Z racji charakterystyki pliku .tsv, gdzie separator jest znakiem tabulacji, w metodzie read_csv ustawiono separator jako "\t". Dodano również główną kolumnę indeksującą jako PassengerId. Wczytanie danych:

In [39]:
import pandas as pd
import numpy
titanic = pd.read_csv("TitanicMess.tsv", sep="\t", index_col='PassengerId')

Po wczytaniu danych wyświetlimy je w przyjemnym dla oka formacie.

In [40]:
from IPython.display import display
display(titanic)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
...,...,...,...,...,...,...,...,...,...,...,...,...
888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,B42,S,Titanic
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,"23,45",NaN,S,Titanic
890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C148,C,Titanic


Wyświetlone zostaje pierwsze oraz ostatnie 5 wierszy. Można od razu zauważyć, że zbiór posiada wartości NaN (null) w kolumnach Age oraz Cabin. 
Poniższa funkcja pozwoli sprawdzić, ile wartości NaN znajduje się w całym zbiorze:

In [41]:
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         173
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

Jak można zauważyć, brakuje 173 wartości w kolumnie Age, 685 wartości w kolumnie Cabin oraz 2 wartości w kolumnie Embarked. 
Sprawdźmy jakie są typy danych w poszczególnych kolumnach:

In [42]:
titanic.dtypes

Survived     int64
Pclass       int64
Name        object
Sex         object
Age         object
SibSp        int64
Parch        int64
Ticket      object
Fare        object
Cabin       object
Embarked    object
ship        object
dtype: object

Część z kolumn posiada typ object, chociaż w niektórych przypadkach typ mógłby być mniej ogólny.
Warto ustalić sobie nową zmienną pomocniczą do wczytanego zbioru, na której będą przeprowadzane wszystkie operacje optymalizujące:

In [43]:
titanic_optimized = titanic

##### Kolumna Sex

Pierwsza zoptymalizowana zostanie kolumna **Sex**, która pomoże w optymalizacji oraz uzupełnieniu danych kolumny **Age**. Kolumna **Sex** powinna zawierać wartości **male** oraz **female** z racji tego, że dane w tej kolumnie dostarczają informacji na temat płci:

In [44]:
print(titanic_optimized['Sex'].unique())

['male' 'female' 'malef' 'mal' 'fem' 'femmale']


Niestety w wartościach znajdują się literówki. Poprawmy wszystkie wartości:

In [45]:
for index, row in titanic_optimized.iterrows():
    sex = str(row["Sex"])
    if sex == "malef" or sex == "mal":
        titanic_optimized.at[index, "Sex"] = "male"
    elif sex == "fem" or sex == "femmale":
        titanic_optimized.at[index, "Sex"] = "female"

Funkcję unique() warto wykonać ponownie z celu upewnienia się, że jedyne wartości jakie występują to **male** oraz **female**:

In [46]:
print(titanic_optimized['Sex'].unique())

['male' 'female']


##### Kolumna Age
Druga zoptymalizowana oraz uzupełniona zostanie kolumna **Age**.

Sprawdźmy, czy wartości można poprawnie sparsować do wartości liczbowej:

In [47]:
wrongValues = 0
for index, row in titanic_optimized.iterrows():
    try:
        value = float(row["Age"])
    except ValueError:
        wrongValues += 1
        
    
print("Niepoprawne wartości: " + str(wrongValues))

Niepoprawne wartości: 25


Niepoprawnych wartości jest w sumie 25. Aby umożliwić optymalizację oraz uzupełnienie danych, wartości niekompatybilne zostaną zastąpione wartością NaN w następujący sposób:

In [48]:
for index, row in titanic_optimized.iterrows():
    try:
        value = float(row["Age"])
    except ValueError:
        titanic_optimized.at[index, "Age"] = float("NaN")

Po tej operacji można ponownie uruchomić sprawdzenie, czy wartości są już poprawnie sparsowane:

In [49]:
wrongValues = 0
for index, row in titanic_optimized.iterrows():
    try:
        value = float(row["Age"])
    except ValueError:
        wrongValues += 1
        
    
print("Niepoprawne wartości: " + str(wrongValues))

Niepoprawne wartości: 0


Liczba wynikowa wynosi 0 także wszystko się zgadza. 

Sprawdźmy teraz poprawność danych, które mamy zapisane. Zacznijmy od wieku najstarszej osoby:

In [50]:
maxAge = 0
for index, row in titanic_optimized.iterrows():
    value = float(row["Age"])
    if value > maxAge:
        maxAge = value
        
    
print("Najstarsza osoba liczy sobie " + str(maxAge) + " lat.")

Najstarsza osoba liczy sobie 4435.0 lat.


Wiek 4435 lat znacznie wykracza poza długość życia rekordzistów, gdzie najdłużej żyjąca kobieta żyła 122 lata i 164 dni a najdłużej żyjący mężczyzna liczył 116 lat i 45 dni. Sprawdźmy jeszcze najmłodszą osobę:

In [51]:
minAge = 4435
for index, row in titanic_optimized.iterrows():
    value = float(row["Age"])
    if value != float("NaN") and value < minAge:
        minAge = value
        
    
print("Najmłodsza osoba liczy sobie " + str(minAge) + " lat.")

Najmłodsza osoba liczy sobie -12.0 lat.


Wartość ujemna w wieku również jest ciekawa. Załóżmy, że osoba na statku niezależnie od płci musi mieć przynajmniej roczek oraz, że maksymalny wiek kobiety to 122 lata a maksymalny wiek mężczyzny to 116 lat. Pozostały wiek traktujemy jako wartość niepoprawną oraz przypisujemy jej wartość NaN:

In [52]:
for index, row in titanic_optimized.iterrows():
    age = float(row["Age"])
    sex = str(row["Age"])
    if age < 1 or age > 122 or (age > 116 and sex == "male"):
        titanic_optimized.at[index, "Age"] = float("NaN")

W tej chwili można z pełną świadomością ustawić typ danych kolumny Age na float oraz potwierdzić zmianę typu danych:

In [53]:
titanic_optimized[["Age"]] = titanic_optimized[["Age"]].astype(float)
titanic.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare         object
Cabin        object
Embarked     object
ship         object
dtype: object

W tym momencie można obliczyć średnią wieku wszystkich pasażerów w zależności od płci, gdzie wartość ta posłuży do uzupełnienia brakujących danych:

In [54]:
sumOfMaleAge = 0
maleCount = 0
for index, row in titanic_optimized.iterrows():
    age = float(row["Age"])
    sex = str(row["Sex"])
    if sex == "male" and age > 0:
        sumOfMaleAge = sumOfMaleAge + age
        maleCount += 1
averageMaleAge = sumOfMaleAge / maleCount
print(averageMaleAge)

30.900692840646652


W zaokrągleniu:

In [55]:
averageMaleAge = round(averageMaleAge)
print(averageMaleAge)

31


Analogiczna obliczenia dla kobiet:

In [56]:
sumOfFemaleAge = 0
femaleCount = 0
for index, row in titanic_optimized.iterrows():
    age = float(row["Age"])
    sex = str(row["Sex"])
    if sex == "female" and ~numpy.isnan(age):
        sumOfFemaleAge += age
        femaleCount += 1
averageFemaleAge = sumOfFemaleAge / femaleCount
print(averageFemaleAge)

27.8984375


W zaokrągleniu:

In [57]:
averageFemaleAge = round(averageFemaleAge)
print(averageFemaleAge)

28


Uzupełnienie wartości:

In [58]:
for index, row in titanic_optimized.iterrows():
    age = float(row["Age"])
    sex = str(row["Sex"])
    if sex == "female" and numpy.isnan(age):
        titanic_optimized.at[index, "Age"] = averageFemaleAge
    elif sex == "male" and numpy.isnan(age):
        titanic_optimized.at[index, "Age"] = averageMaleAge

Na potwierdzenie optymalizacji oraz uzupełnienia wartości:

In [59]:
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

##### Kolumna Ship

Wartości w kolumnie **Ship** zawierają te same wartości - **Titanic**. W kwestii optymalizacji, można usunąć tę kolumnę, ponieważ nic nie zmienia.

In [60]:
titanic_optimized.drop('ship', axis=1, inplace=True)
display(titanic)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,"23,45",NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30,C148,C


##### Kolumna Cabin

Sytuacja z kolumną **Cabin** jest podobna do sytuacji z kolumną **ship**. Nie wnosi ona za dużo do ogólnego rozrachunku i dodatkowo brakuje w niej znacznej większości danych. W tym przypadku również zostanie usunięta:

In [61]:
titanic_optimized.drop('Cabin', axis=1, inplace=True)
display(titanic)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",S
...,...,...,...,...,...,...,...,...,...,...
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,"23,45",S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30,C


##### Kolumna Embarked

W przypadku kolumny Embarked, problemem były dwie brakujące wartości. Sprawdźmy najpierw, jakie wartości są przechowywane w tej kolumnie:

In [62]:
print(titanic_optimized['Embarked'].unique())

['S' 'C' 'Q' 'So' nan 'Co' 'Qe']


Zapiszmy wartości w tablicy z pomnięciem wartości pustych:

In [63]:
embarkedValues = ['S', 'C', 'Q', "So", 'Co', 'Qe']

Uzupełnijmy wartości:

In [64]:
for index, row in titanic_optimized.iterrows():
    embarked = str(row["Embarked"])
    if embarked == "nan":
        titanic_optimized.at[index, "Embarked"] = 'S'

Sprawdzenie pustych wartości:

In [65]:
titanic.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

Wszystkie wartości są uzupełnione.

##### Kolumna Fare

W przypadku kolumny **Fare** warto ustawić odpowiedni typ danych zwłaszcza, że dane przechowywane w tej kolumnie powinny być tylko wartościami liczbowymi. Jedynym problemem jest to, że wartości dziesiętne przechowywane są po przecinku a nie po kropce. Sprawdźmy, czy wartości są poprawne a jak nie, to ustawmy je jako null:

In [66]:
for index, row in titanic_optimized.iterrows():
    value = str(row["Fare"])
    try:
        convertedValue = float(value.replace(',','.'))
        titanic_optimized.at[index, "Fare"] = convertedValue
    except ValueError:
        titanic_optimized.at[index, "Fare"] = float("NaN")

Sprawdźmy puste wartości:

In [67]:
titanic.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        1
Embarked    0
dtype: int64

Po zmianie sposobu przedstawienia wartości pokazała się jedna pusta wartość. Obliczmy średnią i przypiszmy wartość średnią w tą pustą wartość:

In [68]:
sumOfFare = 0
fareCount = 0
for index, row in titanic_optimized.iterrows():
    fare = float(row["Fare"])
    if ~numpy.isnan(fare):
        sumOfFare += fare
        fareCount += 1
averageFare = sumOfFare / fareCount
print(averageFare)

32.09233928170592


W zaokrągleniu:

In [69]:
averageFare = round(averageFare)
print(averageFare)

32


Można uzupełnić brakującą wartość:

In [70]:
for index, row in titanic_optimized.iterrows():
    fare = float(row["Fare"])
    if numpy.isnan(fare):
        titanic_optimized.at[index, "Fare"] = averageFare

Sprawdzenie wartości:

In [71]:
titanic.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

Konwersja na typ float:

In [72]:
titanic_optimized[["Fare"]] = titanic_optimized[["Fare"]].astype(float)
titanic.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
dtype: object

##### Kolumna PassengerId

W ostatniej części sprawdźmy, czy nigdzie nie ma duplikatów w **PassengerId**. W tym celu skorzystam z biblioteki pandas:

In [73]:
titanic_optimized[titanic_optimized.duplicated(keep=False)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4.0,1,1,PP 9549,16.7,S
11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4.0,1,1,PP 9549,16.7,S
11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4.0,1,1,PP 9549,16.7,S
225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38.0,1,0,19943,90.0,S
225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38.0,1,0,19943,90.0,S


Pojawiły się duplikaty dla ID 11 i 225. Usuńmy więc te powtarzające się wiersze:

In [74]:
titanic_optimized.drop_duplicates(keep = False, inplace = True)

Potwierdzenie usunięcia duplikatów:

In [75]:
titanic_optimized[titanic_optimized.duplicated(keep=False)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,


##### Podsumowanie

Zoptymalizowano dane w kolumnach Sex, Age, Ship, Embarked, Fare oraz PassengerId.

##### Wyeksportowanie pliku

Ostatnim zadaniem jest wyeksportowanie oczyszczonego pliku, również w formacie **.tsv**.

In [76]:
titanic_optimized.to_csv('TitanicCleaned.tsv', sep = '\t')